<a href="https://colab.research.google.com/github/rameshveer/ML_Algorithms/blob/master/Deep_Convolutional_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Convolutional GANs

In [2]:
# import the libraries

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Setting some hyperparameters
# We set the size of the batch

batchSize = 64

# We set the size of the generated images (64x64)

imageSize = 64

In [4]:
# Creating the transformations
# We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images

transform = transforms.Compose([transforms.Scale(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [5]:
# Loading the dataset
# We download the training set in the ./data folder and we apply the previous transformations on each image

dataset = dset.CIFAR10(root = './data', download = True, transform = transform)

# We use dataLoader to get the images of the training set batch by batch

dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2)


Extracting ./data/cifar-10-python.tar.gz to ./data


In [6]:
#checking the availability of cuda devices
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
ngpu = 1

In [8]:
# Defining the weights_init function that takes as input a neural network m and that will initialize all its weights

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [9]:
# Defining the generator
# We introduce a class to define the generator
# We introduce the __init__() function that will define the architecture of the generator
# We inherit from the nn.Module tools
# We create a meta module of a neural network that will contain a sequence of modules 
#(convolutions, full connections, etc.)
# We start with the first inversed convolution
# We normalize all the features along the dimension of the batch
# We apply a ReLU rectification to break the linearity
# We add the second later of inversed convolution
# We normalize again
# We apply the ReLU activation function
# We add the third layer of inversed convolution
# We normalize again
# We apply the ReLU activation function
# We add the fourth layer of inversed convolution
# We normalize again
# We apply the ReLU activation function
# We add the fifth layer of inversed convolution
# We apply a Tanh rectification to break the linearity and stay between -1 and +1

class G(nn.Module):
    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
            nn.Tanh()
        )

# We define the forward function that takes as argument an input that will be fed to the neural network, 
# and that will return the output containing the generated images
# We forward propagate the signal through the whole neural network of the generator defined by self.main
# We return the output containing the generated images

    def forward(self, input):
        output = self.main(input)
        return output

In [10]:
# Creating the generator

# We create the generator object
# We initialize all the weights of its neural network

netG = G().to(device)
netG.apply(weights_init)

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [11]:
# Defining the discriminator
# We introduce a class to define the discriminator
# We introduce the __init__() function that will define the architecture of the discriminator
# We inherit from the nn.Module tools
# We create a meta module of a neural network 
# that will contain a sequence of modules (convolutions, full connections, etc.)
# We start with a convolution
# We apply a LeakyReLU
# We add another convolution
# We normalize all the features along the dimension of the batch
# We apply another LeakyReLU
# We add another convolution
# We normalize again
# We apply another LeakyReLU
# We add another convolution
# We normalize again

class D(nn.Module):
    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 512, 4, 2, 1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(512, 1, 4, 1, 0, bias = False),
            nn.Sigmoid()
        )
        
    def forward(self, input):
        output = self.main(input)
        return output.view(-1)

In [12]:
# Creating the discriminator

netD = D().to(device)
netD.apply(weights_init)

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [13]:
# Training the DCGANs

criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))

In [37]:
import os
os.mkdir('results')
#os.mkdir('weights')

In [20]:
fixed_noise = torch.randn(128, 100, 1, 1, device=device)
real_label = 1
fake_label = 0

niter = 5
g_loss = []
d_loss = []

In [29]:
for epoch in range(niter):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label, device=device)

        output = netD(real_cpu)
        errD_real = criterion(output, label.float())
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, 100, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach())
        errD_fake = criterion(output, label.float())
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label.float())
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        
        #save the output
        if i % 100 == 0:
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f' % (epoch, niter, i, len(dataloader), errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
            vutils.save_image(real_cpu,'results/real_samples.png',normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(),'results/fake_samples_epoch_%03d.png' % (epoch),normalize=True)
    
    # Check pointing for every epoch
    torch.save(netG.state_dict(), 'weights/netG_epoch_%d.pth' % (epoch))
    torch.save(netD.state_dict(), 'weights/netD_epoch_%d.pth' % (epoch))

[0/25][0/782] Loss_D: 0.3823 Loss_G: 2.4721 D(x): 0.7704 D(G(z)): 0.0715 / 0.1259
[0/25][100/782] Loss_D: 0.5873 Loss_G: 2.4181 D(x): 0.6754 D(G(z)): 0.0270 / 0.1440
[0/25][200/782] Loss_D: 0.4405 Loss_G: 3.0477 D(x): 0.7883 D(G(z)): 0.1376 / 0.0634
[0/25][300/782] Loss_D: 0.1446 Loss_G: 3.8112 D(x): 0.9061 D(G(z)): 0.0320 / 0.0320
[0/25][400/782] Loss_D: 0.3110 Loss_G: 3.8767 D(x): 0.8750 D(G(z)): 0.1321 / 0.0297
[0/25][500/782] Loss_D: 0.4187 Loss_G: 4.5060 D(x): 0.8105 D(G(z)): 0.1352 / 0.0287
[0/25][600/782] Loss_D: 0.5305 Loss_G: 4.6225 D(x): 0.9176 D(G(z)): 0.2768 / 0.0285
[0/25][700/782] Loss_D: 0.7537 Loss_G: 1.7981 D(x): 0.6009 D(G(z)): 0.0756 / 0.2262
[1/25][0/782] Loss_D: 0.6031 Loss_G: 3.7065 D(x): 0.9297 D(G(z)): 0.3488 / 0.0366
[1/25][100/782] Loss_D: 0.6035 Loss_G: 2.4837 D(x): 0.6430 D(G(z)): 0.0563 / 0.1285
[1/25][200/782] Loss_D: 0.2676 Loss_G: 3.5650 D(x): 0.8693 D(G(z)): 0.1055 / 0.0398
[1/25][300/782] Loss_D: 2.1485 Loss_G: 7.9254 D(x): 0.9810 D(G(z)): 0.8271 / 0.0

In [43]:
# To copy results folder from Colab to G Drive folder

!cp -r /content/results /content/drive/My\ Drive/GANS/